In [1]:
import awkward as ak
import numpy as np
import pandas as pd
import json
import os
import shutil
import pathlib
from typing import List, Optional
import pyarrow.parquet as pq

from coffea import processor
from coffea.nanoevents.methods import candidate, vector
from coffea.analysis_tools import Weights, PackedSelection

import warnings
warnings.filterwarnings("ignore", message="Found duplicate branch ")

import pyarrow as pa
import pyarrow.parquet as pq
import pickle as pkl

In [2]:
def pad_val(
    arr: ak.Array,
    value: float,
    target: int = None,
    axis: int = 0,
    to_numpy: bool = False,
    clip: bool = True,
):
    """
    pads awkward array up to ``target`` index along axis ``axis`` with value ``value``,
    optionally converts to numpy array
    """
    if target:
        ret = ak.fill_none(ak.pad_none(arr, target, axis=axis, clip=clip), value, axis=None)
    else:
        ret = ak.fill_none(arr, value, axis=None)
    return ret.to_numpy() if to_numpy else ret


from collections import defaultdict
def dsum(*dicts):
    ret = defaultdict(int)
    for d in dicts:
        for k, v in d.items():
            ret[k] += v
    return dict(ret)  

In [141]:
class HwwProcessor(processor.ProcessorABC):
    def __init__(self, year="2017", yearmod="", channels=["ele","mu","had"], output_location="./"):
        self._year = year
        self._yearmod = yearmod
        self._channels = channels
        self._output_location = output_location
        
        # define variables to save for each channel
        self._skimvars = {
            'ele': [
                "lepton_pt",
                "lep_isolation",
                "met",
                "ht",
                "mt_lep_met",
                "dr_jet_candlep",
            ],
            'mu': [
                "lepton_pt",
                "lep_isolation",
                "met",
                "ht",
                "mt_lep_met",
                "dr_jet_candlep",
                "mu_mvaId"
            ],
            'had': [
                "fatjet_pt",
            ],
        }

        
        # trigger paths
        self._HLTs = {
            2016: {
                'ele': [
                    "Ele27_WPTight_Gsf",
                    "Ele115_CaloIdVT_GsfTrkIdT",
                    "Photon175",
                    # "Ele50_CaloIdVT_GsfTrkIdT_PFJet165", # extra                                                                                                                                                                                                              
                    # "Ele15_IsoVVVL_PFHT600", # VVL                                                                                                                                                                                                                            
                ],
                'mu': [
                    "Mu50",
                    "TkMu50",
                    "IsoMu24",
                    "IsoTkMu24",
                    # "Mu55",                                                                                                                                                                                                                                                   
                    # "Mu15_IsoVVVL_PFHT600" # VVL                                                                                                                                                                                                                              
                ],
                'had': [
                    "PFHT800",
                    "PFHT900",
                    "AK8PFJet360_TrimMass30",
                    "AK8PFHT700_TrimR0p1PT0p03Mass50",
                    "PFHT650_WideJetMJJ950DEtaJJ1p5",
                    "PFHT650_WideJetMJJ900DEtaJJ1p5",
                    "PFJet450",
                ],
            },
            2017: {
                'ele': [
                    "Ele35_WPTight_Gsf",
                    "Ele115_CaloIdVT_GsfTrkIdT",
                    "Photon200",
                    # "Ele50_CaloIdVT_GsfTrkIdT_PFJet165", # extra                                                                                                                                                                                                              
                    # "Ele15_IsoVVVL_PFHT600", # VVL                                                                                                                                                                                                                            
                ],
                'mu': [
                    "Mu50",
                    "IsoMu27",
                    "OldMu100",
                    "TkMu100",
                    # "Mu15_IsoVVVL_PFHT600", # VVL                                                                                                                                                                                                                             
                ],
                'had': [
                    "PFHT1050",
                    "AK8PFJet400_TrimMass30",
                    "AK8PFJet420_TrimMass30",
                    "AK8PFHT800_TrimMass50",
                    "PFJet500",
                    "AK8PFJet500",
                ],
            },
            2018: {
                'ele': [
                    "Ele32_WPTight_Gsf",
                    "Ele115_CaloIdVT_GsfTrkIdT",
                    "Photon200",
                    # "Ele50_CaloIdVT_GsfTrkIdT_PFJet165", # extra                                                                                                                                                                                                              
                    # "Ele15_IsoVVVL_PFHT600", # VVL                                                                                                                                                                                                                            
                ],
                'mu': [
                    "Mu50",
                    "IsoMu24",
                    "OldMu100",
                    "TkMu100",
                    # "Mu15_IsoVVVL_PFHT600", # VVL                                                                                                                                                                                                                             
                ],
                'had': [
                    "PFHT1050",
                    "AK8PFJet400_TrimMass30",
                    "AK8PFJet420_TrimMass30",
                    "AK8PFHT800_TrimMass50",
                    "PFJet500",
                    "AK8PFJet500",
                ],
            }
        }[int(self._year)]
    
        # https://twiki.cern.ch/twiki/bin/view/CMS/MissingETOptionalFiltersRun2                                                                                                                                                                                                 
        self._metfilters = {
            2016: [
                "goodVertices",
                "globalSuperTightHalo2016Filter",
                "HBHENoiseFilter",
                "HBHENoiseIsoFilter",
                "EcalDeadCellTriggerPrimitiveFilter",
                "BadPFMuonFilter",
                "eeBadScFilter",
            ],
            2017: [
                "goodVertices",
                "globalSuperTightHalo2016Filter",
                "HBHENoiseFilter",
                "HBHENoiseIsoFilter",
                "EcalDeadCellTriggerPrimitiveFilter",
                "BadPFMuonFilter",
                # "BadChargedCandidateFilter",                                                                                                                                                                                                                                  
                "eeBadScFilter",
                "ecalBadCalibFilter",
            ],
            2018:  [
                "goodVertices",
                "globalSuperTightHalo2016Filter",
                "HBHENoiseFilter",
                "HBHENoiseIsoFilter",
                "EcalDeadCellTriggerPrimitiveFilter",
                "BadPFMuonFilter",
                # "BadChargedCandidateFilter",                                                                                                                                                                                                                                  
                "eeBadScFilter",
                "ecalBadCalibFilter",
            ],
        }[int(self._year)]
        
        # https://twiki.cern.ch/twiki/bin/viewauth/CMS/BtagRecommendation                                                                                                                                                                                                       
        self._btagWPs = {
            '2016preVFP': {
                'loose': 0.0508,
                'medium': 0.2598,
                'tight': 0.6502,
            },
            '2016postVFP': {
                'loose': 0.0480,
                'medium': 0.2489,
                'tight': 0.6377,
            },
            '2017': {
                'loose': 0.0532,
                'medium': 0.3040,
                'tight': 0.7476,
            },
            '2018': {
                'loose': 0.0490,
                'medium': 0.2783,
                'tight': 0.7100,
            },
        }[year+yearmod]
        
        self.selections = {}
        self.cutflows = {}
        
    @property
    def accumulator(self):
        return self._accumulator
    
    def save_dfs_parquet(self, fname, dfs_dict, ch, isMC, dataset, sumgenweight, cutflows):
        if self._output_location is not None:
            table = pa.Table.from_pandas(dfs_dict)
            pq.write_table(table, './outfiles/' + ch + '/parquet/' + fname + '.parquet')            
            
            if isMC:
                metadata = dict(sumgenweight=sumgenweight, year=self._year, mc=isMC, dataset=dataset, cutflow=cutflows[ch])  
            else:
                metadata = dict(year=self._year, mc=isMC, dataset=dataset, cutflows=cutflows)
                
            if not os.path.exists('./outfiles/' + ch + '/' + ch + '_metadata.pkl'):
                file = open('./outfiles/' + ch + '/'  + ch + '_metadata.pkl', 'wb')
                pkl.dump(metadata, file)  
            else:
                file = open('./outfiles/' + ch + '/'  + ch + '_metadata.pkl', 'rb')
                metadata_all = pkl.load(file)          

                metadata_all['cutflow'] = (dsum(metadata_all['cutflow'] , metadata['cutflow']))

                file = open('./outfiles/' + ch + '/'  + ch + '_metadata.pkl', 'wb')
                pkl.dump(metadata_all, file) 
                
        
    def ak_to_pandas(self, output_collection: ak.Array) -> pd.DataFrame:
        output = pd.DataFrame()
        for field in ak.fields(output_collection):
            output[field] = ak.to_numpy(output_collection[field])
        return output
     
        
    def add_selection(self, name: str, sel: np.ndarray, channel: list = None):
        """Adds selection to PackedSelection object and the cutflow dictionary"""
        channels = channel if channel else self._channels
        for ch in channels:
            self.selections[ch].add(name, sel)
            self.cutflows[ch][name] = np.sum(self.selections[ch].all(*self.selections[ch].names))
        
    def process(self, events: ak.Array):
        """Returns skimmed events which pass preselection cuts and with the branches listed in self._skimvars"""
        dataset = events.metadata['dataset']
        isMC = hasattr(events, "genWeight")
        sumgenweight = ak.sum(events.genWeight) if isMC else 0
        nevents = len(events)
                      
        # empty selections and cutflows
        self.selections = {}
        self.cutflows = {}
        for ch in self._channels:
            self.selections[ch] = PackedSelection()
            self.cutflows[ch] = {}
            self.cutflows[ch]["all"] = nevents

        # trigger                                                                                                                                                                                                   
        triggers = {}
        for ch in self._channels:
            if ch=="had" and isMC:
                trigger = np.ones(nevents, dtype='bool')
            else:
                # apply trigger to both data and MC (except for hadronic channel)                                                                                                                                   
                trigger = np.zeros(len(events), dtype='bool')
                for t in self._HLTs[ch]:
                    if t in events.HLT.fields:
                        trigger = trigger | events.HLT[t]
            self.add_selection("trigger", trigger, [ch])
            del trigger
            
        # metfilters
        metfilters = np.ones(nevents, dtype='bool')
        for mf in self._metfilters:
            if mf in events.Flag.fields:
                metfilters = metfilters & events.Flag[mf]
        self.add_selection("metfilters", metfilters)

        
        # define muon objects    
        loose_muons = (
                    ( ((events.Muon.pt > 30) & (events.Muon.pfRelIso04_all < 0.25)) |
                      (events.Muon.pt > 55) )
                    & (np.abs(events.Muon.eta) < 2.4)
                    & (events.Muon.looseId)
                )
        n_loose_muons = ak.sum(loose_muons, axis=1)
        
        good_muons = (
                    (events.Muon.pt > 28)
                    & (np.abs(events.Muon.eta) < 2.4)
                    & (np.abs(events.Muon.dz) < 0.1)
                    & (np.abs(events.Muon.dxy) < 0.05)
                    & (events.Muon.sip3d <= 4.0)
                    & events.Muon.mediumId
                )
        n_good_muons = ak.sum(good_muons, axis=1)
    
        # define electron objects    
        loose_electrons = (
                    ( ((events.Electron.pt > 38) & (events.Electron.pfRelIso03_all < 0.25)) |
                      (events.Electron.pt > 120) )
                    & ((np.abs(events.Electron.eta) < 1.44) | (np.abs(events.Electron.eta) > 1.57))
                    & (events.Electron.cutBased >= events.Electron.LOOSE)
                )
        n_loose_electrons = ak.sum(loose_electrons, axis=1)
            
        good_electrons = (
                    (events.Electron.pt > 38)
                    & ((np.abs(events.Electron.eta) < 1.44) | (np.abs(events.Electron.eta) > 1.57))
                    & (np.abs(events.Electron.dz) < 0.1)
                    & (np.abs(events.Electron.dxy) < 0.05)
                    & (events.Electron.sip3d <= 4.0)
                    & (events.Electron.mvaFall17V2noIso_WP90)
                )
        n_good_electrons = ak.sum(good_electrons, axis=1)        

        # leading lepton
        goodleptons = ak.concatenate([events.Muon[good_muons], events.Electron[good_electrons]], axis=1)
        goodleptons = goodleptons[ak.argsort(goodleptons.pt, ascending=False)]               
        candidatelep = ak.firsts(goodleptons)
        
        ### candidate leptons
        candidatelep_p4 = ak.zip(
            {
                "pt": candidatelep.pt,
                "eta": candidatelep.eta,
                "phi": candidatelep.phi,
                "mass": candidatelep.mass,
                "charge": candidatelep.charge,
            },
            with_name="PtEtaPhiMCandidate",
            behavior=candidate.behavior,
        )           
    
        # relative isolation
        lep_reliso = candidatelep.pfRelIso04_all if hasattr(candidatelep, "pfRelIso04_all") else candidatelep.pfRelIso03_all

        # mini isolation
        mu_miso = candidatelep.miniPFRelIso_all
        
        # MVA-ID
        mu_mvaId = candidatelep.mvaId if hasattr(candidatelep, "mvaId") else np.zeros(nevents)
        
        
        # FATJETS
        fatjets = events.FatJet
        candidatefj = fatjets[    
                    (fatjets.pt > 200)
                    & (abs(fatjets.eta) < 2.5)
                    & fatjets.isTight
                    # & fatjets.puId==7   #### TODO field not found
                ]
        candidatefj_lep = ak.firsts(candidatefj[ak.argmin(candidatefj.delta_r(candidatelep_p4),axis=1,keepdims=True)])        
        
        # deltaR
        dr_jet_candlep = candidatefj_lep.delta_r(candidatelep_p4) 
        
        # MET
        met = events.MET
        mt_lep_met = np.sqrt(
                     2.*candidatelep_p4.pt*met.pt*(ak.ones_like(met.pt) - np.cos(candidatelep_p4.delta_phi(met)))
                     )  
        # JETS
        goodjet = events.Jet[
                    (events.Jet.pt > 30) 
                    & (abs(events.Jet.eta) < 2.5) 
                    & events.Jet.isTight
                ]
        ht = ak.sum(goodjet.pt, axis=1)        

        ### event selections 
        self.add_selection(
                    name='leptonKin',
                    sel=(candidatelep.pt > 30),
                    channel=['mu']
                )
        self.add_selection(
                    name='oneLepton',
                    sel=(n_good_muons == 1) & (n_good_electrons == 0) & (n_loose_electrons == 0),
                    channel=['mu']
                )
        self.add_selection('leptonIsolation', sel=(
                    ((candidatelep.pt > 30)
                    & (candidatelep.pt < 55)
                    & (lep_reliso < 0.25)
                    )
                    | ((candidatelep.pt >= 55 )
                    & (candidatelep.miniPFRelIso_all < 0.2)) 
        ),channel=['mu'])
        self.add_selection('leptonInJet', sel=(dr_jet_candlep < 0.8), channel=['mu', 'ele'])
        self.add_selection('ht', sel=(ht > 200), channel=['mu', 'ele'])    
        self.add_selection('mt', sel=(mt_lep_met < 100), channel=['mu', 'ele'])    
  
    
        ### selections for electrons
        self.add_selection(
                    name='leptonKin',
                    sel=(candidatelep.pt > 40),
                    channel=['ele']
                )
        self.add_selection(
                    name='oneLepton',
                    sel=(n_good_muons == 0) & (n_loose_muons == 0) & (n_good_electrons == 1),
                    channel=['ele']
                )   
        self.add_selection('leptonIsolation', sel=(
                    ((candidatelep.pt > 30)
                    & (candidatelep.pt < 120)
                    & (lep_reliso < 0.3)
                    )
                    | ((candidatelep.pt >= 120 )
                    & (candidatelep.miniPFRelIso_all < 0.2)) 
        ),channel=['ele'])    
    
    
        #### TODO add selection for had
        
        
        
        
        
        
        
        ### TODO different names per job, and 1 parquet file per channel
    
        # initialize pandas dataframe
        output = {}
        for ch in self._channels:
            out = {}
            for var in self._skimvars[ch]:
                if var == "lepton_pt": 
                    value = pad_val(candidatelep.pt,0)
                    out[var] = value
                if var == "dr_jet_candlep":
                    value = pad_val(dr_jet_candlep,0)
                    out[var] = value                   
                if var == "mt_lep_met":
                    value = pad_val(mt_lep_met,0)                    
                    out[var] = value             
                if var == "ht":
                    value = pad_val(ht,0)
                    out[var] = value         
                if var == "met":
                    value = pad_val(met.pt,0)
                    out[var] = value                       
                if var == "lep_isolation": 
                    value = pad_val(lep_reliso,-1)                       
                    out[var] = value
                else: continue
                
            # print arrays and selections to debug
            # print(out)
            # print(selections[ch].all(*selections[ch].names))
            
            # apply selections
            output[ch] = {
                key: value[self.selections[ch].all(*self.selections[ch].names)] for (key, value) in out.items()
            }

            # convert arrays to pandas
            if not isinstance(output[ch], pd.DataFrame): output[ch] = self.ak_to_pandas(output[ch])
                
        # now save pandas dataframes
        fname = events.behavior["__events_factory__"]._partition_key.replace("/", "_")      
        fname = 'condor_' + fname
        for ch in self._channels:
            if not os.path.exists('./outfiles/' + ch):     ### creating a directory for each channel
                os.makedirs('./outfiles/' + ch)
            if not os.path.exists('./outfiles/' + ch + '/parquet'):     ### creating a directory for each channel
                os.makedirs('./outfiles/' + ch + '/parquet')
            self.save_dfs_parquet(fname, output[ch], ch, isMC, dataset, sumgenweight, self.cutflows)
        
        return {}

    def postprocess(self, accumulator):
        return accumulator

In [153]:
import json
datasets = {"GluGluHToWWToLNuQQ_M125_TuneCP5_PSweight_13TeV-powheg2-jhugen727-pythia8": "HWW",
           }

fileset = {}
for dataset_name,dataset in datasets.items():
    print(dataset_name)
    with open("../fileset/v2_2/2017.json", 'r') as f:
        files = json.load(f)[dataset][dataset_name]
    
    files = [files[0], files[1]]
    
    # use all_files False if you want to test
    all_files = True
    # need to define the fileset but call them with xcache
    if all_files:
        fileset[dataset_name] = ["root://xcache/"+ f for f in files]
    else:
        fileset[dataset_name] = [["root://xcache/"+ f for f in files][1]]

GluGluHToWWToLNuQQ_M125_TuneCP5_PSweight_13TeV-powheg2-jhugen727-pythia8


In [155]:
import uproot
uproot.open.defaults['xrootd_handler'] = uproot.source.xrootd.MultithreadedXRootDSource

from coffea.processor import IterativeExecutor,Runner,DaskExecutor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

dask_executor = False

channels=["ele","mu", "had"]
    
# define executor 
if dask_executor:
    executor = DaskExecutor(compression=1, status=True, client=client, treereduction=2)
else:
    executor = IterativeExecutor(compression=1, status=True)

# define the runner (Same as before)
run = Runner(executor=executor,savemetrics=True,chunksize=10000,schema=NanoAODSchema)

# run
for dataset,dataset_files in fileset.items():
    new_fileset = {dataset: dataset_files}
    print(dataset)
    hwwproc = HwwProcessor(year="2017", channels=channels, output_location="./")
    out,metrics = run(new_fileset,'Events',processor_instance=hwwproc)

### merge parquet
for ch in channels:
    data = pd.read_parquet('./outfiles/' + ch + '/parquet')
    data.to_parquet('./outfiles/' + ch + '/' + ch + '_df.parquet')

GluGluHToWWToLNuQQ_M125_TuneCP5_PSweight_13TeV-powheg2-jhugen727-pythia8


Processing:   0%|          | 0/20 [00:00<?, ?chunk/s]

In [150]:
# debug output
import pandas as pd
data = pq.read_table('./outfiles/ele/ele_df.parquet')
data = data.to_pandas()
data

,lepton_pt,lep_isolation,met,ht,mt_lep_met,dr_jet_candlep
0,75.976952,0.071459,37.473038,542.25000,17.162954,0.246079
1,199.699066,0.011814,67.740738,753.34375,95.632385,0.165259
2,184.322098,0.308283,50.377277,680.03125,57.140987,0.075172
3,82.911263,0.000000,30.618563,588.18750,52.946758,0.321030
4,61.240463,0.009035,69.222122,512.31250,21.472057,0.277764
...,...,...,...,...,...,...
69,150.018997,0.073323,51.497971,397.25000,94.927094,0.004264
70,115.633339,0.000000,119.576302,524.28125,68.676880,0.242941
71,123.703903,0.596314,36.989067,583.28125,4.887266,0.085734
72,68.639893,0.022911,163.772659,688.03125,12.427414,0.293719


In [152]:
# debug output
with open('./outfiles/ele/ele_metadata.pkl', 'rb') as f:
    metadata = pkl.load(f)
metadata

{'sumgenweight': 284249.38,
 'year': '2017',
 'mc': True,
 'dataset': 'GluGluHToWWToLNuQQ_M125_TuneCP5_PSweight_13TeV-powheg2-jhugen727-pythia8',
 'cutflow': {'all': 98400,
  'trigger': 5552,
  'metfilters': 5551,
  'leptonInJet': 109,
  'ht': 109,
  'mt': 78,
  'leptonKin': 76,
  'oneLepton': 75,
  'leptonIsolation': 74}}